# **Pipeline run 2**

## **cDBGs Generations**

```bash
# exit when any command fails
set -e

# keep track of the last executed command
trap 'last_command=$current_command; current_command=$BASH_COMMAND' DEBUG
# echo an error message before exiting
trap 'echo "\"${last_command}\" command filed with exit code $?."' EXIT

start=`date +%s`


DATA_DIR=/home/mabuelanin/Desktop/kexpression_experiment/symbolic/data/drtamer_data

echo "Generating cDBG for raw reads (two files) with k=25"
/usr/bin/time -v bcalm -kmer-size 25 -max-memory 12000 -out SRR11015356_before_k25 -in list_reads &> bcalm_before_25.log

echo "Generating cDBG for raw reads (two files) with k=75"
/usr/bin/time -v bcalm -kmer-size 75 -max-memory 12000 -out SRR11015356_before_k75 -in list_reads &> bcalm_before_75.log

echo "CD-HIT-EST Clustering for SRR11015356_before_k75"
/usr/bin/time -v cd-hit-est -i SRR11015356_before_k75.unitigs.fa -n 11 -c 0.95 -o clusters_SRR11015356_before_k75 -d 0 -T 0 -M 12000 &> cdhit_SRR11015356_before_k75.log
echo "Exporting representative sequences only from the CDHIT*clst and for SRR11015356_before_k75.unitigs.fa ..."

cat clusters_SRR11015356_before_k75.clstr | grep "\*" | awk -F"[>.]" '{print ">"$2}' | grep -Fwf - -A1 <(seqkit seq -w 0 SRR11015356_before_k75.unitigs.fa) | grep -v "^\-\-" > reps_unitigs_SRR11015356_before_k75.fa

echo "Creating cDBG for reps_unitigs_SRR11015356_before_k75.fa with k=25"
/usr/bin/time -v bcalm -kmer-size 25 -max-memory 12000 -out reps_unitigs_SRR11015356_before_k75_after_k25.fa -in reps_unitigs_SRR11015356_before_k75.fa  -abundance-min 1 &> bcalm_before_75_after_25.log

echo "Creating cDBG for reps_unitigs_SRR11015356_before_k75.fa with k=75"
/usr/bin/time -v bcalm -kmer-size 75 -max-memory 12000 -out reps_unitigs_SRR11015356_before_k75_after_k75.fa -in reps_unitigs_SRR11015356_before_k75.fa  -abundance-min 1 &> bcalm_before_75_after_75.log


end=`date +%s`

runtime=$((end-start))

echo "DONE SUCCESSFULLY in ${runtime}"

```

## **Coverage calculations**

```bash
<<NOTES
- https://www.biostars.org/p/262240/#262362

- Bowtie2, on the other hand, is designed to map reads continuously to the indexed reference.
  It just happens to be particularly good at dealing with multi-mapping reads, which is why people like using it to align to the transcriptome.

However, if you're interested in doing quantification with a known transcriptome in a generally well-annotated organism like mouse, 
I'd recommend doing quantification of the transcripts directly (e.g. using Salmon). 
If you then want to do DE, this can be followed with something like tximport to get results into your favorite DE tool (e.g. DESeq2, EdgeR, limma-voom).
NOTES

# exit when any command fails
set -e

# keep track of the last executed command
trap 'last_command=$current_command; current_command=$BASH_COMMAND' DEBUG
# echo an error message before exiting
trap 'echo "\"${last_command}\" command filed with exit code $?."' EXIT

## ----------------------------------------------------

# 1. Aligning the "reps_unitigs_SRR11015356_before_k75.fa" on the "extractedGTF_gencode.v33.transcripts.fa"

## 1.1 Indexing 
#bowtie2-build /home/mabuelanin/Desktop/kexpression_experiment/symbolic/reports/report5_gencodeReprs/extractedGTF_gencode.v33.transcripts.fa bowtie2_extractedGTF_gencode.v33.transcripts
#samtools faidx /home/mabuelanin/Desktop/kexpression_experiment/symbolic/reports/report5_gencodeReprs/extractedGTF_gencode.v33.transcripts.fa # Important for IGV Visualization

## 1.1 Aligning
bowtie2 -x bowtie2_extractedGTF_gencode.v33.transcripts -f reps_unitigs_SRR11015356_before_k75.fa -S bowtie2_reps_unitigs_SRR11015356_before_k75.sam

## 1.2 Converting to sorted BAM
samtools view -S -b bowtie2_reps_unitigs_SRR11015356_before_k75.sam -o bowtie2_reps_unitigs_SRR11015356_before_k75.bam
samtools sort bowtie2_reps_unitigs_SRR11015356_before_k75.bam -o sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam

## 1.3 Generating coverage histogram
samtools index sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam
samtools depth sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam > sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam.cov
cat sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam.cov | awk -F'\t' '{print $3}' | sort -n | uniq -c > sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam.cov.hist

## 1.4 [Optional] View the alignment in the terminal
# samtools tview sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam extractedGTF_gencode.v33.transcripts.fa

## ---------------------------------------------------

# 2. Aligning the "reps_unitigs_SRR11015356_before_k75_after_k75.fa.unitigs.fa" on the "extractedGTF_gencode.v33.transcripts.fa"

## 2.1 Indexing [Done in step #1]
# bowtie2-build extractedGTF_gencode.v33.transcripts.fa bowtie2_extractedGTF_gencode.v33.transcripts
# samtools faidx extractedGTF_gencode.v33.transcripts.fa # Important for IGV Visualization


## 2.1 Aligning
bowtie2 -x bowtie2_extractedGTF_gencode.v33.transcripts -f reps_unitigs_SRR11015356_before_k75_after_k75.fa.unitigs.fa -S reps_unitigs_SRR11015356_before_k75_after_k75.sam


## 2.2 Converting to sorted BAM
samtools view -S -b reps_unitigs_SRR11015356_before_k75_after_k75.sam -o bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam
samtools sort bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam -o sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam


## 2.3 Generating coverage histogram
samtools index sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam
samtools depth sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam > sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam.cov
cat sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam.cov | awk -F'\t' '{print $3}' | sort -n | uniq -c > sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam.cov.hist


## 2.4 [Optional] View the alignment in the terminal
# samtools tview sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam extractedGTF_gencode.v33.transcripts.fa


## ---------------------------------------------------

# 3. Merging the two histograms

paste sorted_bowtie2_reps_unitigs_SRR11015356_before_k75.bam.cov.hist sorted_bowtie2_reps_unitigs_SRR11015356_before_k75_after_k75.bam.cov.hist > before_after_cov.hist

# 4. IGV Visualization

## 4.1 Indexing the unitigs files


<<ALIGN_STATS
1.
2528169 reads; of these:
  2528169 (100.00%) were unpaired; of these:
    2381769 (94.21%) aligned 0 times
    38856 (1.54%) aligned exactly 1 time
    107544 (4.25%) aligned >1 times
5.79% overall alignment rate
----
2.
1993007 reads; of these:
  1993007 (100.00%) were unpaired; of these:
    1895443 (95.10%) aligned 0 times
    25712 (1.29%) aligned exactly 1 time
    71852 (3.61%) aligned >1 times
4.90% overall alignment rate
ALIGN_STATS
```

## **Coverage [before75 - after75]**
```tsv
    BEFORE     AFTER
    
  16073 0	  13892 0
7981185 1	6899381 1
2208998 2	1538820 2
 776057 3	 483184 3
 303413 4	 192052 4
 131201 5	  83912 5
  60766 6	  43640 6
  29814 7	  21261 7
  15593 8	  12213 8
   8846 9	   6827 9
   5344 10	   4545 10
   3566 11	   2884 11
   2432 12	   2003 12
   1815 13	   1452 13
   1132 14	    787 14
    793 15	    465 15
    576 16	    412 16
    416 17	    394 17
    219 18	    192 18
    203 19	     61 19
    130 20	     57 20
    117 21	     10 21
     69 22	      3 22
     18 23	      8 23
      6 24	      4 24
      3 25	      8 25
     10 26	     13 26
      6 27	     12 27
      3 28	      7 28
     12 29	     28 29
     25 30	      2 30
      4 31	      6 31
      1 32	      2 33
      1 33	      5 34
      2 34	      4 35
      3 35	      3 36
      5 36	      1 37
      5 37	      3 38
      1 38	     11 39
      2 39	      4 40
      6 40	      3 41
      7 41	      4 42
      9 42	      1 43
      7 43	      4 44
      5 44	      6 45
      4 45	      4 46
      1 46	      7 47
      7 47	      4 48
      7 48	

```

---

## **IGV Visualization sample**

- Target: extractedGTF_transcriptome
- Track1: Before75 Unitigs
- Track2: Before75_After75 Unitigs

![](./igv_before75_after75_sample.png)
